In [ ]:
%%capture
! pip install demoji

In [ ]:
%%capture
! pip install indic-nlp-library

In [ ]:
%%capture
! pip install transformers

In [ ]:
%%capture
! pip install sentencepiece

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/MarathiSentiment")

model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/MarathiSentiment")

Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/981 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134M [00:00<?, ?B/s]

In [ ]:
import json 
import pandas as pd
import os
import demoji
import re
import string
from transformers import pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
os.chdir("/content/drive/My Drive/HASOC-2021/Subtask1-Marathi")

In [ ]:
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

In [ ]:
def remove(text):
  filtered = ""
  for i in text.split():
    if i not in stop_words:
      filtered = filtered + i + " "
  return filtered[:-1]

def remove_stop(df):
  df['text'] = df.text.apply(remove)
  return df

def score_row(text):
  out = classifier(text)[0]
  negative = out[0]['score']
  positive = out[2]['score']
  return positive-negative

def senti(df):
  df['polarity'] = df.text.apply(score_row)
  return df

def profane_row(text):
  count_profane = 0
  tokens = text.split()
  for i in tokens:
    if i in profane_words:
      count_profane = count_profane + 1
  return count_profane/len(tokens)

def frac_profane(df):
  df['frac_profane'] = df.text.apply(profane_row)
  return df

def remove_punctuation(text):
    return ''.join([c for c in text if c not in string.punctuation])

def remove_url(text):
    return re.sub(r'https?://\S+', ' ', text)

def remove_mention(text):
    return re.sub(r'@[A-Za-z0-9_]+', ' ', text)

def remove_hashtag(text):
    return re.sub(r'.*?(#\w+)|.+', ' ', text)  #TODO: check if using/removing the hashtag has any improvement

def remove_emoji(text):
    return demoji.replace(text, ' ')

def clean_row(text):
    return ' '.join(remove_punctuation(remove_url(remove_mention(remove_emoji(text)))).split()).lower()

def clean(df):
    df['text'] = df.text.apply(clean_row)
    return df

def normalize_row(text):
  return normalizer.normalize(text)

def normalize(df):
  df['text'] = df.text.apply(normalize_row)
  return df

In [ ]:
profane_words = {
  'अकराम्हशी',
  'अकराम्हशीचा',
  'अकराम्हशीच्या',
  'आंद्या',
  'आंद्याचा',
  'आंद्याच्या',
  'आंद्यात',
  'आईघाला',
  'आईघाल्',
  'आईघाल्या',
  'आईघाल्याचा',
  'आईजवाडा',
  'आईजवाडाचा',
  'आईझव',
  'आईझवली',
  'आईझवलीचा',
  'आईझवाडा',
  'आईझवाडाचा',
  'कँडल',
  'कँडलचा',
  'कँडलच्या',
  'गांड',
  'गांडचा',
  'गांडच्या',
  'गांडीचा',
  'गांडीत',
  'गांडू',
  'गांडूचा',
  'गांडूच्या',
  'गांडूत',
  'गोट्या',
  'गोट्याचा',
  'गोट्याच्या',
  'गोट्यात',
  'चीनाल',
  'चीनालचा',
  'चीनालच्या',
  'चुत',
  'चुतचा',
  'चुतच्या',
  'चुतत',
  'चुतमारीचा',
  'चुतमारीच्या',
  'चुतमारीच्यात',
  'च्यायला',
  'छिनाल',
  'छिनालचा',
  'छिनालच्या',
  'झवली',
  'झवलीचा',
  'झवलीत',
  'झवाड्या',
  'झवाड्याचा',
  'झवाड्याच्या',
  'झाटू',
  'झाटूचा',
  'झाटूच्या',
  'झाटूत',
  'पुच्ची',
  'पुच्चीचा',
  'पुच्चीच्या',
  'पुच्चीत',
  'फोकणीच्या',
  'फोकणीच्याचा',
  'फोकणीच्यात',
  'फोद्री',
  'फोद्रीचा',
  'फोद्रीच्या',
  'फोद्रीच्यात',
  'फोद्रीत',
  'बावळट',
  'बावळटच्या',
  'बावळटत',
  'बुडाला',
  'बुल्ली',
  'बुल्लीचा',
  'बुल्लीच्या',
  'बुल्लीत',
  'बोचा',
  'बोचाच्या',
  'बोचात',
  'भडवा',
  'भडवाच्या',
  'भडविच्याभिकारचोट',
  'भडव्या',
  'भडव्यात',
  'भुंड्',
  'भुंड्त',
  'भुंड्यात',
  'भुंड्यातत',
  'भोक',
  'भोकचा',
  'भोकत',
  'भोकाच्या',
  'भोसडा',
  'भोसडाचा',
  'भोसडात',
  'भोसडीच्या',
  'भोसडीच्यात',
  'माईचा',
  'माईचात',
  'माईच्या',
  'मादरचोद',
  'मारीच्या',
  'मारीच्यात',
  'मुठ्ठया',
  'मुठ्ठयाचा',
  'मुठ्ठयात',
  'रंडी',
  'रंडीचा',
  'रंडीच्या',
  'रंडीच्यात',
  'रंडीत',
  'रांड',
  'रांडचा',
  'रांडच्या',
  'रांडीच्या',
  'रांडीच्यात',
  'लवड्या',
  'लवड्याचा',
  'लवड्याच्या',
  'लवड्यात'
 }
stop_words = {
  'अधिक',
  'अनेक',
  'अशी',
  'असलयाचे',
  'असलेल्या',
  'असा',
  'असून',
  'असे',
  'आज',
  'आणि',
  'आता',
  'आपल्या',
  'आला',
  'आली',
  'आले',
  'आहे',
  'आहेत',
  'एक',
  'एका',
  'कमी',
  'करणयात',
  'करून',
  'का',
  'काम',
  'काय',
  'काही',
  'किवा',
  'की',
  'केला',
  'केली',
  'केले',
  'कोटी',
  'गेल्या',
  'घेऊन',
  'जात',
  'झाला',
  'झाली',
  'झाले',
  'झालेल्या',
  'टा',
  'डॉ',
  'तर',
  'तरी',
  'तसेच',
  'ता',
  'ती',
  'तीन',
  'ते',
  'तो',
  'त्या',
  'त्याचा',
  'त्याची',
  'त्याच्या',
  'त्याना',
  'त्यानी',
  'त्यामुळे',
  'त्री',
  'दिली',
  'दोन',
  'न',
  'नाही',
  'निर्ण्य',
  'पण',
  'पम',
  'परयतन',
  'पाटील',
  'म',
  'मात्र',
  'माहिती',
  'मी',
  'मुबी',
  'म्हणजे',
  'म्हणाले',
  'म्हणून',
  'या',
  'याचा',
  'याची',
  'याच्या',
  'याना',
  'यानी',
  'येणार',
  'येत',
  'येथील',
  'येथे',
  'लाख',
  'व',
  'व्यकत',
  'सर्व',
  'सागित्ले',
  'सुरू',
  'हजार',
  'हा',
  'ही',
  'हे',
  'होणार',
  'होत',
  'होता',
  'होती',
  'होते'
  } 

In [ ]:
classifier = pipeline('sentiment-analysis', model='l3cube-pune/MarathiSentiment', return_all_scores=True)

In [ ]:
# Normalization
factory=IndicNormalizerFactory()
normalizer=factory.get_normalizer("mr")

In [ ]:
df = pd.read_csv('./Data/hasoc2021_mr_test-blind-2021.csv')

In [ ]:
df.head()

,text_id,text
0,hasoc_mr_1875,माणसाची सर्वात मोठी समस्या इतिहासाचे दाखले द...
1,hasoc_mr_1876,मुद्दामहून थुकत बसला ते पण दोस्तांना बोलवून म...
2,hasoc_mr_1877,तुझे नोटिफिकेशन ऑन केलेत परंतु नाव दिसलं का शि...
3,hasoc_mr_1878,पांडुरंग हरी वासुदेव हरी तु तुझ्या घरी मी म...
4,hasoc_mr_1879,भेंचोद जेव्हा कोरोना कोरोना चालू होतं तेव्हा ...


In [ ]:
df = clean(df)
df = remove_stop(df)
df = normalize(df)
df = frac_profane(df)



In [ ]:
df.head()

,text_id,text,frac_profane
0,hasoc_mr_1875,माणसाची सर्वात मोठी समस्या इतिहासाचे दाखले देत...,0.0
1,hasoc_mr_1876,मुद्दामहून थुकत बसला दोस्तांना बोलवून म्हणुन झ...,0.0
2,hasoc_mr_1877,तुझे नोटिफिकेशन ऑन केलेत परंतु नाव दिसलं शिव्य...,0.0
3,hasoc_mr_1878,पांडुरंग हरी वासुदेव हरी तु तुझ्या घरी माझ्या ...,0.0
4,hasoc_mr_1879,भेंचोद जेव्हा कोरोना कोरोना चालू होतं तेव्हा आ...,0.0


In [ ]:
df = senti(df)

In [ ]:
df.to_csv('./Data/blind_test_pre_processed.csv')

In [ ]:
df2 = pd.read_csv("")